# 1. 準備環境

## 1.1 匯入檔案處理與圖像顯示模組

In [3]:
import os # 處理檔案和目錄
import glob # 查找符合特定模式的所有路徑名

# 在 Jupyter Notebook 或 IPython 環境中顯示圖像
from IPython.display import Image, display
from IPython import display

In [4]:
# 清理輸出區域，這樣在重新執行程式時，不會看到先前的輸出結果。
# 這有助於保持 Notebook 的整潔，避免混淆舊的與新的輸出內容。
display.clear_output()

## 1.2 定義工作目錄

我們可以使用 `os.getcwd()` 取得**當前工作目錄的路徑**，並將其儲存到 `HOME` 變數中，並且把它印出來。

在深度學習專案中，確定當前工作目錄很重要，因為接下來的檔案讀寫操作都會基於這個目錄進行。

In [5]:
HOME = os.getcwd()
print(HOME)

/kaggle/working


我們可以執行 shell 指令 `nvidia-smi` 指令來查看 GPU 的狀態。

`nvidia-smi` 這個指令會顯示目前系統中可用的 NVIDIA GPU 資訊，包括 GPU 型號、使用的驅動版本、當前的記憶體使用情況等。

對於訓練深度學習模型，了解 GPU 的狀態非常重要，因為它會直接影響模型訓練的效能。

In [6]:
# 查看 GPU 的狀態。
!nvidia-smi

Tue Aug 20 02:29:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

在 IPython 或 Jupyter Notebook 中，
- `!` 符號用來執行 shell 指令，這相當於直接在終端機中輸入指令。
- `%` 符號用來執行 IPython 的 magic commands，不只包含呼叫 shell 指令，還有更高層次的操作。

## 1.3 安裝並匯入 `ultralytics`

[`ultralytics`](https://pypi.org/project/ultralytics/) 是 YOLOv8 的官方實作版本。

In [7]:
# 用 pip 安裝 ultralytics
# --quiet 選項能減少安裝過程中的輸出訊息，使 pip 只會顯示必要的錯誤或警告。
!pip install --quiet ultralytics

# 匯入 ultralytics
import ultralytics

[`ultralytics.checks()`](https://docs.ultralytics.com/reference/utils/checks/) 會檢查軟硬體配置（例如確認是否有正確版本的 PyTorch 以及 GPU 支援等），這樣可以確保 YOLOv8 模型順利運行。

In [8]:
# 查當前的系統環境是否適合 YOLOv8 運行
ultralytics.checks()

Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.7/8062.4 GB disk)


假設你的系統已成功安裝並配置了 YOLOv8，應該會顯示：

```
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.7/8062.4 GB disk)
```
具體情況如下：

1. **Ultralytics YOLOv8.2.78**：這表示你安裝的 YOLOv8 版本是 8.2.78。
2. **Python-3.10.13**：你正在使用 Python 3.10.13 版本。
3. **torch-2.1.2**：你安裝的 PyTorch 版本是 2.1.2，這是 YOLOv8 依賴的重要深度學習框架。
4. **CUDA:0 (Tesla T4, 15095MiB)**：這表示系統檢測到一張 NVIDIA Tesla T4 GPU，並且這張 GPU 擁有 15095MB 的顯示卡記憶體。CUDA 是用來加速 GPU 計算的技術。
5. **Setup complete**：這表示 YOLOv8 的環境配置已經完成，可以開始使用。
6. **4 CPUs, 31.4 GB RAM, 5771.7/8062.4 GB disk**：這表示你的系統有 4 個 CPU，31.4 GB 的記憶體，以及 8062.4 GB 的硬碟空間，其中 5771.7 GB 已經被使用。

這個結果確認了你的系統環境配置良好，適合進行 YOLOv8 模型的訓練與推理。


## 1.4 安裝 `roboflow` 並載入資料集

**Roboflow** 是一個提供影像資料集管理與標註工具的平台。我們用 Roboflow 的 API （即 [`roboflow`](https://pypi.org/project/roboflow/) 套件）來載入現成的手語資料集。

首先在當前工作目錄下創建一個名為 "datasets" 的資料夾。這個資料夾將用來存放我們即將下載和處理的資料集。

In [9]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [10]:
# 用 pip 安裝 roboflow
!pip install --quiet roboflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


下載資料集的流程如下。

1. **取得 API 金鑰**：首先，你需要在 Roboflow 的平台上註冊並創建一個專案。註冊後，你可以在帳戶設定或專案設定中找到你的 API 金鑰。

2. **使用 API 金鑰進行身份驗證**：在你的程式中，你需要將 API 金鑰傳遞給 Roboflow 的 API，以便進行身份驗證。例如，rf = Roboflow(api_key="你的API金鑰") 這行程式碼就負責這項操作。

3. **下載資料集**：一旦身份驗證成功，你就可以使用 API 來下載專案中的資料集。Roboflow 會根據你的請求，將資料集以你指定的格式（例如 YOLOv5 格式）打包並下載到你的工作環境中。

In [11]:
from roboflow import Roboflow

# 初始化 Roboflow，並使用 API 金鑰進行身份驗證。
rf = Roboflow(api_key="vQpY4GwAhAtrSZQTm2J0")

# 取得指定的工作區和專案。
# 這份資料集是來自 David Lee 的工作區，專案名稱為 american-sign-language-letters。
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")

# 從專案中下載資料集的版本 1，並指定格式為 "yolov5"。
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to American-Sign-Language-Letters-1 in yolov5pytorch:: 100%|██████████| 3458/3458 [00:00<00:00, 9737.26it/s]


## 1.5 準備資料集配置檔案

`dataset.location` 是一個包含下載資料集存放位置的變數。就我們載入的資料集而言，此變數的值是 `/kaggle/working/datasets/American-Sign-Language-Letters-1`。

In [12]:
# 切換到資料集所在的目錄
%cd {HOME}
%cd {dataset.location}

/kaggle/working
/kaggle/working/datasets/American-Sign-Language-Letters-1


現在的目錄底下有一個名為 `data.yaml` 的檔案，它是 YOLOv5 的配置檔案，通常包含資料集的相關資訊，例如類別、訓練和驗證集的位置等。

我們使用 `!cat` 指令來顯示 `data.yaml` 檔案的內容。

In [13]:
# 顯示配置檔案的內容，包含類別、訓練和驗證集的位置等。
!cat {dataset.location}/data.yaml

names:
- A
- B
- C
- D
- E
- F
- G
- H
- I
- J
- K
- L
- M
- N
- O
- P
- Q
- R
- S
- T
- U
- V
- W
- X
- Y
- Z
nc: 26
train: American-Sign-Language-Letters-1/train/images
val: American-Sign-Language-Letters-1/valid/images


這段結果顯示了資料集的 `data.yaml` 配置檔案內容，並指出資料集包含以下幾個主要部分：

1. **`names`**：列出了資料集中所有的類別標籤，從 A 到 Z，共 26 個字母。這表示資料集的目標是辨識美國手語中的26個字母。
   
2. **`nc`**：表示資料集的類別數目，即 26 個字母。

3. **`train`**：指向訓練集影像的路徑 `American-Sign-Language-Letters-1/train/images`，這是模型訓練時使用的資料。

4. **`val`**：指向驗證集影像的路徑 `American-Sign-Language-Letters-1/valid/images`，這是模型訓練過程中用來評估效能的資料。


我們要在 `data.yaml` 檔案的末尾加入一行 `path: .`，這明確指定資料集的根目錄位置。
- `echo` 指令的作用是將 "path: ." 這行文字輸出。
- 這項輸出經由管道符號 `|` 傳給下一個指令。
- `tee -a` 的作用是同時將輸出顯示在終端，並寫入指定檔案中，不會覆蓋檔案的現有內容。


In [14]:
!echo "path: ." | tee -a data.yaml

path: .


In [15]:
!cat {dataset.location}/data.yaml

names:
- A
- B
- C
- D
- E
- F
- G
- H
- I
- J
- K
- L
- M
- N
- O
- P
- Q
- R
- S
- T
- U
- V
- W
- X
- Y
- Z
nc: 26
train: American-Sign-Language-Letters-1/train/images
val: American-Sign-Language-Letters-1/valid/images
path: .


觀察最後一行，`path: .` 應改已經加進去了。

## 1.6 使用 Ngrok 將 TensorBoard 公開到外網

[**TensorBoard**](https://www.tensorflow.org/tensorboard?hl=zh-tw) 是 TensorFlow 提供的視覺化工具，旨在幫助開發者監控和分析機器學習模型的訓練過程。透過 TensorBoard，使用者可以查看模型的損失、精度、學習率等各種訓練指標，並且可以可視化神經網絡結構、觀察輸入數據和張量的分佈情況，甚至監控模型參數的變化過程。這些功能有助於開發者理解模型的訓練進度，調整超參數，並改善模型性能。

在 TensorFlow 框架下進行訓練時，TensorBoard 的使用非常簡單——幾乎不用額外配置，只需在訓練過程中添加回調函數（callback）即可。

只要下一行簡單的 shell 指令
```shell
tensorboard --logdir=runs
```
然後，你可以在瀏覽器中打開 http://localhost:6006 查看即時的訓練指標。

雖然 YOLO 模型更接近 PyTorch 框架，但由於 PyTorch 提供了與 TensorBoard 整合的支持，所以使用者只需導入 TensorBoard 函式庫並設置 `SummaryWriter`，然後將訓練指標寫入到指定的日誌目錄。

不過，更大的問題是，當我們在 Kaggle 的雲端環境中運行 TensorBoard 時，無法像在本機上那樣直接訪問 TensorBoard 的介面。為了解決這個問題，我們可以使用 **Ngrok** 這樣的工具，將 TensorBoard 的介面公開到網路上，這樣就能夠方便地從瀏覽器中訪問並監控訓練過程。

**[Ngrok](https://ngrok.com/)** 是一種反向代理（reverse proxy）工具，它可以讓你將本地主機（localhost）正在運行的服務公開到外網（Wide Area Network）上，讓你能夠從外部訪問這個服務，所以很適合在開發和測試中使用。



- 使用 `wget` 指令從網路下載 ngrok 的 Linux 版本壓縮包 (tgz 格式)。  
- 使用 `tar` 指令解壓縮剛剛下載的 ngrok 壓縮包。`xvzf` 是 `tar` 指令的選項，分別代表解壓（x）、顯示過程（v）、解壓縮 gzip 文件（z）、指定文件（f）。

In [16]:
%cd {HOME} 

# 下載 Ngrok 的壓縮包 (tgz 格式)。
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# 解壓縮
!tar xvzf ngrok-v3-stable-linux-amd64.tgz

--2024-08-20 02:30:25--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8974299 (8.6M) [application/octet-stream]
Saving to: 'ngrok-v3-stable-linux-amd64.tgz'

ngrok-v3-stable-lin 100%[===================>]   8.56M  29.7MB/s    in 0.3s    

2024-08-20 02:30:26 (29.7 MB/s) - 'ngrok-v3-stable-linux-amd64.tgz' saved [8974299/8974299]

ngrok


`authtoken` 是 Ngrok 提供的一個指令，用來配置你的帳戶金鑰，這樣你就可以使用 ngrok 的服務了。請注意，這裡的 `authtoken` 是用來連接你的 ngrok 帳戶，你應該替換為你自己的金鑰。

In [17]:
# 執行解壓縮後的 ngrok 可執行檔，進行身份驗證。
!./ngrok authtoken 2kHBHpxSXjxl0CpAB1voiE5Pqm8_3ystxDW9bBnpySQuxo6ZT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from subprocess import Popen
from os import chmod
from os.path import isfile
import json
import time
import psutil

def launch_tensorboard():
    tb_process, ngrok_process = None, None

    # 啟動 TensorBoard
    # 如果 TensorBoard 尚未啟動，則啟動它
    if not is_process_running('tensorboard'):
        tb_command = 'tensorboard --logdir /kaggle/working/runs/ --host 0.0.0.0 --port 6006 --load_fast=false'
        tb_process = run_cmd_async_unsafe(tb_command)
    
    # 創建 ngrok 隧道並印出其公共 URL
    # 如果 ngrok 尚未啟動，則啟動它以創建一個 HTTP 隧道
    if not is_process_running('ngrok'):
        ngrok_process = run_cmd_async_unsafe('./ngrok http http://localhost:6006')
        time.sleep(1)  # 等待 ngrok 啟動隧道
    ngrok_api_res = urlopen('http://127.0.0.1:4040/api/tunnels', timeout=10)
    ngrok_api_res = json.load(ngrok_api_res)
    
    # 確保 ngrok 隧道已經創建
    assert len(ngrok_api_res['tunnels']) > 0, 'ngrok tunnel not found'
    tb_public_url = ngrok_api_res['tunnels'][0]['public_url']
    print(f'TensorBoard URL: {tb_public_url}')

    return tb_process, ngrok_process

# 非同步執行命令，雖不安全但簡單
def run_cmd_async_unsafe(cmd):
    return Popen(cmd, shell=True)

# 檢查指定名稱的程序是否正在運行
def is_process_running(process_name):
    running_process_names = (proc.name() for proc in psutil.process_iter())
    return process_name in running_process_names

# 啟動 TensorBoard 並建立 ngrok 隧道
launch_tensorboard()


TensorBoard URL: https://4d1f-35-193-231-61.ngrok-free.app


(<Popen: returncode: None args: 'tensorboard --logdir /kaggle/working/runs/ -...>,
 <Popen: returncode: None args: './ngrok http http://localhost:6006'>)

TensorBoard 2.15.1 at http://0.0.0.0:6006/ (Press CTRL+C to quit)


簡要說明：

- **launch_tensorboard() 函式**：這個函式啟動 TensorBoard 並通過 ngrok 創建一個公共 URL，使其能夠從外部訪問。這在 Kaggle Notebook 或其他雲端環境中特別有用，因為這些環境通常沒有直接的外部訪問權限。
  
- **run_cmd_async_unsafe() 函式**：這個輔助函式用來非同步執行 shell 命令。儘管它是「不安全」的，因為沒有進行額外的錯誤檢查，但它在簡單的情境下仍然有效。

- **is_process_running() 函式**：這個函式用來檢查特定名稱的程序是否正在運行，通過使用 `psutil` 函式庫來檢查系統中當前正在運行的所有程序。

這段程式碼主要用於在遠程或受端環境中啟動 TensorBoard 並使其通過 ngrok 隧道公開，使得你可以從外部監控訓練過程。

# 2. 訓練模型


接下來正式進入模型的訓練。我們將會執行以下 shell 指令。

```shell
!yolo device=0,1 task=detect mode=train model=yolov8l.pt optimizer="auto" batch=32 data={dataset.location}/data.yaml epochs=50 imgsz=800
```

它含有大量的關鍵字參數（keyword argument），我們在執行前先來做個詳細解釋。


## 2.1 YOLO 指令參數解析


### **`device=0,1`** 指定了要使用的加速裝置
這裡表示使用 GPU 0 和 1 來加速訓練。

Kaggle 在 Session options 中免費提供「GPU T4 x2」、「GPU P100」、「TPU VM v3-8」等加速裝置，以下是它們的比較以及它們在 YOLO 模型訓練中的適用性分析。

#### 比較與適用性分析

1. **計算性能**:
   - **TPU v3-8** 在 FP16（半精度浮點數）性能上遙遙領先，適合大規模的深度學習模型訓練和快速迭代。
   - **Tesla T4 x2** 提供的雙卡配置能有效利用多 GPU 訓練並加速計算過程，特別是在分布式訓練中。
   - **Tesla P100** 雖然沒有 Tensor 核心，但在 FP32 精度下依然表現強勁，適合需要更高精度訓練的任務。

2. **記憶體大小**:
   - **TPU v3-8** 擁有 128 GB 的 HBM 記憶體，適合處理極大規模的模型和資料集。
   - **Tesla T4 x2** 的總記憶體為 32 GB，足夠訓練大多數 YOLO 模型。
   - **Tesla P100** 擁有 16 GB HBM2 記憶體，對於中型模型是足夠的，但可能不足以處理更大的模型或更高解析度的圖像。

3. **軟體支援**:
   - **TPU v3-8** 需要使用 TensorFlow 或 JAX 進行訓練，並可能需要對 YOLO 模型進行一些修改和優化。
   - **Tesla T4** 和 **P100** 支援更廣泛的深度學習框架，包括 PyTorch、TensorFlow、MXNet 等，這使得它們更加靈活，尤其是在使用 PyTorch 的情況下。

#### 結論：哪一個最適合 YOLO 模型訓練？

- **Tesla T4 x2** 是 YOLO 模型訓練的最佳選擇，特別是對於需要利用多 GPU 進行加速的情況。它的雙 GPU 配置能夠提供足夠的計算能力和記憶體來訓練大型模型，如 YOLOv8l 和 YOLOv8x，並且與 PyTorch 和 TensorFlow 等框架高度相容。

- **Tesla P100** 可以用於中型模型的訓練，如 YOLOv8m 或 YOLOv8l，當計算資源有限且不需要 Tensor 核心加速時，P100 也是一個不錯的選擇。

- **TPU v3-8** 適合非常大規模的訓練任務，特別是當使用 TensorFlow 或 JAX 進行開發時。然而，由於需要進行適配且對於 YOLO 這類 CNN 模型，GPU 可能更適合，因此 TPU 不一定是最理想的選擇。

根據這些分析，**Tesla T4 x2** 是你在 Kaggle 上進行 YOLO 模型訓練的最佳選擇。

### **`task=detect`** 表示執行物件偵測任務。
- **`detect` 是物件偵測**，也是 YOLO 的核心任務，用於從影像或視頻中檢測並標註出各類物件的邊界框和類別標籤。
- **`segment` 是語義分割**，它將影像中的每個像素分類到不同物件或背景中。YOLO 在這個任務中不僅檢測物件的邊界框，還提供每個物件的精確像素遮罩。
- **`classify` 是圖像分類**，也就是整張影像分配到預定義的類別之一。YOLO 在這個模式下會忽略物件的空間位置，只關心影像中是否存在某個類別的物件。
- **`pose` 是姿態估計**，可進行人體姿態估計，識別並標注人體的各個關鍵點（如肩膀、肘部、膝蓋等），從而推測人體的姿態。
- **`track` 是物件追蹤**，它結合物件偵測與追蹤技術，用於在影片序列中對物件進行持續追蹤。這個任務可以跟隨特定物件在多幀影像中的移動軌跡。


### **`mode=train`** 表示訓練模式。
- YOLO 提供了多種模式來應對不同的任務需求，包括 `train`（訓練）、`val`（驗證）、`predict`（預測）、`export`（匯出）、`track`（追蹤）和 `benchmark`（基準測試）。
- **`train` 是訓練模式**，用於從頭開始或基於預訓練模型進行模型訓練。
    - 在這個模式下，YOLO 模型會通過反向傳播算法（通常是基於梯度下降法的優化器，如 SGD 或 Adam）學習訓練數據中的特徵，調整模型參數，使得模型能夠準確地執行目標任務，如物件偵測或分類。
    - 此外，YOLO 會根據配置的超參數（如學習率、批次大小、增強技術等）進行多個 epoch 的訓練，每個 epoch 都會遍歷整個訓練數據集。
    - 訓練過程中，模型的性能指標（如損失函數、精確度等）會被記錄下來，用於監控訓練進度。
- **`val` 是驗證（validation）模式**，用來驗證集上評估模型的性能。這個模式不會進行訓練，而是通過載入預訓練或訓練過的模型，對驗證數據進行推理，並計算如 mAP（mean Average Precision）、精確度、召回率等指標。`val` 模式通常在訓練過程結束後或訓練過程中間隔性地運行，以檢查模型的表現並確保模型不會過擬合。
- **`predict` 是預測模式**，用於對新數據進行推理和檢測。這個模式使用訓練好的模型，對給定的輸入圖像或影像進行物件檢測，並輸出檢測結果。`predict` 模式適用於實際應用中的模型部署，如對影片串流或靜態圖片進行即時物件檢測。
- **`export` 是匯出模式**，用於將模型轉換成不同的格式以便於部署。這些格式可能包括 **ONNX**、**TensorFlow Lite**（tflite）、**CoreML**、**TorchScript** 等。當需要在不同的推理框架或硬體平台上部署模型時使用 `export` 模式。匯出的模型可以在手機、嵌入式設備、瀏覽器等不同環境中運行。
- **`track` 是追蹤模式**，用來對物件進行跟蹤。這個模式通常與物件偵測結合，能夠在影片或連續圖像中追蹤同一物件的運動。適用於監控系統、自動駕駛、智慧交通等場景，跟蹤在一系列影像中移動的物件。
- **`benchmark` 是基準測試模式**，用於測試模型的性能和資源需求。這個模式會運行一些基準測試來評估模型的推理速度、計算需求、記憶體使用等。`benchmark` 模式常用在部署前測試模型性能，以確保模型在特定硬體環境中的表現符合要求。

### **`model=yolov8l.pt`** 指定了預訓練模型的路徑或名稱
- 這裡使用 YOLOv8 large (yolov8l) 版本。
- YOLOv8 提供了多個不同大小和複雜度的模型版本，以滿足不同應用場景的需求。
- 層數和參數愈多，準確度越高，可以讓模型學習到更複雜的特徵，但也需要越多資源來計算和推理。
- yolov8l 以外的其他選項：YOLOv8n（輕量版）、YOLOv8s（小型版）、YOLOv8m（中型版）、YOLOv8x（超大型版）。

### **`optimizer="auto"`** 讓 YOLOv8 自動選擇最合適的優化器。
- SGD、Adam、AdamW、RMSProp、AdaGrad 都是訓練深度學習模型時常用的優化器。
- 在 YOLOv8 中，預設的優化器通常是 AdamW，因為它在大多數深度學習任務中表現出良好的效果。
- 如果你的模型存在過擬合問題，可以考慮使用 AdamW 或 SGD。如果你的資料集非常大，並且希望更穩定的訓練過程，可以選擇 SGD。

### **`batch=32`** 設定批次大小為 32
- 這意味著每次訓練會同時處理 32 張圖片。
- 批次大小會影響 GPU 記憶體的使用和模型的收斂速度。
- 較大的批次通常能帶來更穩定的梯度估計，但也需要更多的記憶體。

### **`data={dataset.location}/data.yaml`** 指定了資料集的配置檔案位置。
- 這裡使用我們之前準備好的 `data.yaml`。
- 在 YOLO 訓練過程中，**資料集配置檔案是必須的**。它告訴模型在哪裡找到訓練集和驗證集、如何處理這些數據、以及模型應該學習哪些類別。

### **`epochs=50`** 設定訓練迭代次數為 50。
- 訓練的 epoch 會影響最終的模型性能和訓練時間。
- 另外，在 YOLO 模型的訓練過程中，**早停策略**（Early Stopping）是一種防止過擬合的技術。
    - 它可以在訓練過程中監控模型的性能指標（如驗證集上的損失或精確度），並在模型性能停止改善時提前終止訓練。
    - `patience`、`delta`、`monitor` 等超參數都跟早停策略有關。
- 這些參數應根據資料集大小和模型收斂情況來調整。

### **`imgsz=800`** 設定影像大小為 800 像素
- `imgsz` 是用來指定訓練時將所有輸入圖片調整為統一尺寸的參數，這對於 CNN 模型（如 YOLO）至關重要，因為它使得網絡中的卷積層和池化層能夠一致地處理特徵圖。
- 關於特徵尺度
    - 如果原始圖片過小，放大到 800x800 可能會導致像素失真，從而影響模型學習到的特徵。
    - 如果原始圖片過大，縮小到 800x800 則可能會丟失一些細節，這可能會影響模型對小物件的偵測能力。
    - 所以圖片尺寸必須適當，才有助於模型學習不同大小物件的特徵
- 關於計算量與性能
    - 影像大小直接影響模型的計算量和性能。
    - 更大的影像可以保留更多的細節，但也會增加計算量。


In [ ]:
%cd {HOME} 

!yolo device=0,1 task=detect mode=train model=yolov8l.pt optimizer="auto" batch=32 data={dataset.location}/data.yaml epochs=50 imgsz=800

/kaggle/working
100%|███████████████████████████████████████| 83.7M/83.7M [00:00<00:00, 191MB/s]
Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
                                                      CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/working/datasets/American-Sign-Language-Letters-1/data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=800, save=True, save_period=-1, cache=False, device=(0, 1), workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, vi

上一段程式碼區塊的輸出訊息分別對應了模型訓練過程中的不同步驟，以下章節是每一部分的解釋。

## 2.2 訓練過程(1) — 環境初始化

```txt
/kaggle/working
Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
                                                      CUDA:1 (Tesla T4, 15095MiB)
```

這部分顯示了你當前的工作目錄 `/kaggle/working`，以及 YOLOv8 和相關環境的初始化資訊。特別是：
- 使用的是 YOLOv8.2.79 版本。
- Python 版本是 3.10.13，並且使用的是 torch-2.1.2 (PyTorch)。
- 系統檢測到兩個 Tesla T4 GPU，各自有 15,095 MB 的顯示記憶體（顯示卡記憶體）。

## 2.3 訓練過程(2) — 超參數設置

```txt
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/working/datasets/American-Sign-Language-Letters-1/data.yaml, epochs=50, ...
```

這段描述了 YOLOv8 模型訓練的所有**超參數**，例如前幾項
- `task=detect` 表示執行物件偵測任務。
- `mode=train` 表示進行訓練。
- `model=yolov8l.pt` 是使用的 **YOLOv8 large** 預訓練模型。
- `data=/kaggle/working/datasets/American-Sign-Language-Letters-1/data.yaml` 指定了資料集的配置文件。
- `epochs=50` 設定訓練 50 個 epoch（迭代）。

這些都是我們設定的。另外 YOLO 還有數十個預設的超參數，參見下表。               


### 2.3.1 超參數總表

| 超參數          | 值 | 定義 | 範疇 | 是否為預設 |
|----------------|---|-----|------------------|-----------|
| task           | detect | 任務類型（物件偵測） | 模型設定相關     | Yes |
| mode           | train | 模型模式（訓練模式） | 模型設定相關     | Yes |
| model          | yolov8l.pt | 使用的預訓練模型 | 模型設定相關     | No |
| pretrained     | True | 是否使用預訓練模型 | 模型設定相關     | Yes |
| single_cls     | False | 是否將所有物件視為單一類別進行訓練 | 模型設定相關     | Yes |
| project        | None | 專案名稱 | 模型設定相關     | Yes |
| exist_ok       | False | 是否覆蓋已存在的專案 | 模型設定相關     | Yes |
| epochs         | 50 | 訓練的總迭代次數 | 訓練過程相關     | Yes |
| batch          | 32 | 批次大小 | 訓練過程相關     | Yes |
| optimizer      | auto | 優化器選擇 | 訓練過程相關     | Yes |
| device         | (0, 1) | 使用的設備 | 訓練過程相關     | No |
| workers        | 8 | 工作進程數量 | 訓練過程相關     | Yes |
| amp            | True | 是否使用自動混合精度 | 訓練過程相關     | Yes |
| lr0            | 0.01 | 初始學習率 | 學習率與優化相關 | Yes |
| lrf            | 0.01 | 最終學習率 | 學習率與優化相關 | Yes |
| momentum       | 0.937 | 動量 | 學習率與優化相關 | Yes |
| weight_decay   | 0.0005 | 權重衰減 | 學習率與優化相關 | Yes |
| imgsz          | 800 | 訓練影像的尺寸 | 資料增強與預處理 | Yes |
| mosaic         | 1.0 | 是否使用 Mosaic 增強 | 資料增強與預處理 | Yes |
| auto_augment   | randaugment | 自動增強策略 | 資料增強與預處理 | Yes |
| hsv_h          | 0.015 | 色相調整 | 資料增強與預處理 | Yes |
| hsv_s          | 0.7 | 飽和度調整 | 資料增強與預處理 | Yes |
| hsv_v          | 0.4 | 亮度調整 | 資料增強與預處理 | Yes |
| scale          | 0.5 | 縮放範圍 | 資料增強與預處理 | Yes |
| translate      | 0.1 | 平移範圍 | 資料增強與預處理 | Yes |
| shear          | 0.0 | 切變範圍 | 資料增強與預處理 | Yes |
| flipud         | 0.0 | 上下翻轉機率 | 資料增強與預處理 | Yes |
| fliplr         | 0.5 | 左右翻轉機率 | 資料增強與預處理 | Yes |
| mixup          | 0.0 | 是否使用 Mixup 增強 | 資料增強與預處理 | Yes |
| copy_paste     | 0.0 | 是否使用 Copy-Paste 增強 | 資料增強與預處理 | Yes |
| erasing        | 0.4 | 隨機擦除概率 | 資料增強與預處理 | Yes |
| crop_fraction  | 1.0 | 裁剪比例 | 資料增強與預處理 | Yes |
| iou            | 0.7 | IoU 閾值 | 驗證與推理相關   | Yes |
| max_det        | 300 | 每張影像最大檢測數量 | 驗證與推理相關   | Yes |
| conf           | None | 信心閾值 | 驗證與推理相關   | Yes |
| val            | True | 是否進行驗證 | 驗證與推理相關   | Yes |
| nms            | False | 是否使用非極大值抑制 | 驗證與推理相關   | Yes |
| save           | True | 是否保存模型 | 模型保存與紀錄   | Yes |
| save_dir       | runs/detect/train2 | 保存模型的路徑 | 模型保存與紀錄   | No |
| save_period    | -1 | 模型保存頻率 | 模型保存與紀錄   | Yes |
| save_txt       | False | 是否保存文本結果 | 模型保存與紀錄   | Yes |
| save_conf      | False | 是否保存信心分數 | 模型保存與紀錄   | Yes |
| save_crop      | False | 是否保存裁剪圖像 | 模型保存與紀錄   | Yes |
| save_json      | False | 是否保存 JSON | 模型保存與紀錄   | Yes |
| save_hybrid    | False | 是否保存混合結果 | 模型保存與紀錄   | Yes |
| plots          | True | 是否生成訓練過程的圖表 | 模型保存與紀錄   | Yes |
| name           | train2 | 訓練的名稱 | 模型保存與紀錄   | No |
| patience       | 100 | 訓練早停的耐心次數 | 其他設置         | No |
| deterministic  | True | 是否使用確定性行為 | 其他設置         | Yes |
| seed           | 0 | 隨機數種子 | 其他設置         | Yes |
| resume         | False | 是否從上次中斷的地方繼續訓練 | 其他設置         | Yes |
| freeze         | None | 冷凍層級 | 其他設置         | Yes |
| verbose        | True | 是否顯示詳細訊息 | 其他設置         | Yes |
| cos_lr         | False | 是否使用餘弦退火學習率 | 其他設置         | Yes |
| time           | None | 訓練時間限制 | 其他設置         | Yes |
| cache          | False | 是否快取資料 | 其他設置         | Yes |
| close_mosaic   | 10 | 關閉 mosaic 的 epoch | 其他設置         | No |
| fraction       | 1.0 | 使用的資料比例 | 其他設置         | Yes |
| profile        | False | 是否進行性能分析 | 其他設置         | Yes |
| multi_scale    | False | 是否使用多尺度訓練 | 其他設置         | Yes |
| overlap_mask   | True | 是否使用重疊遮罩 | 其他設置         | No |
| mask_ratio     | 4 | 遮罩比率 | 其他設置         | No |
| dropout        | 0.0 | Dropout 比率 | 其他設置         | Yes |
| split          | val | 驗證集劃分名稱 | 其他設置         | Yes |
| half           | False | 是否使用半精度浮點數 | 其他設置         | Yes |
| dnn            | False | 是否使用深度學習推理加速 | 其他設置         | Yes |
| source         | None | 資料來源 | 其他設置         | Yes |
| vid_stride     | 1 | 影片步長 | 其他設置         | Yes |
| stream_buffer  | False | 是否使用串流緩衝 | 其他設置         | Yes |
| visualize      | False | 是否可視化結果 | 其他設置         | Yes |
| augment        | False | 是否增強資料 | 其他設置         | Yes |
| agnostic_nms   | False | 是否使用類別不可知 NMS | 其他設置         | Yes |
| classes        | None | 目標類別 | 其他設置         | Yes |
| retina_masks   | False | 是否使用 Retina 遮罩 | 其他設置         | Yes |
| embed          | None | 嵌入設置 | 其他設置         | Yes |
| show           | False | 是否顯示訓練過程 | 其他設置         | Yes |
| save_frames    | False | 是否保存影格 | 其他設置         | Yes |
| line_width     | None | 邊界框線寬 | 其他設置         | Yes |
| format         | torchscript | 模型格式 | 其他設置         | Yes |
| keras          | False | 是否啟用 Keras 模式 | 其他設置         | Yes |
| optimize       | False | 是否進行優化 | 其他設置         | Yes |
| int8           | False | 是否使用 int8 量化 | 其他設置         | Yes |
| dynamic        | False | 是否使用動態軟體 | 其他設置         | Yes |
| simplify       | False | 是否簡化模型 | 其他設置         | Yes |
| opset          | None | ONNX 操作集版本 | 其他設置         | Yes |
| workspace      | 4 | 工作空間大小 | 其他設置         | Yes |
| box            | 7.5 | Box 項損失 | 損失函數相關     | Yes |
| cls            | 0.5 | Class 項損失 | 損失函數相關     | Yes |
| dfl            | 1.5 | DFL 項損失 | 損失函數相關     | Yes |
| pose           | 12.0 | Pose 項損失 | 損失函數相關     | Yes |
| kobj           | 1.0 | 物件分類損失 | 損失函數相關     | Yes |
| label_smoothing| 0.0 | 標籤平滑 | 損失函數相關     | Yes |
| nbs            | 64 | 批次大小 | 損失函數相關     | Yes |
| degrees        | 0.0 | 旋轉角度 | 損失函數相關     | Yes |
| perspective    | 0.0 | 透視範圍 | 損失函數相關     | Yes |
| bgr            | 0.0 | 是否使用 BGR 色彩順序 | 損失函數相關     | Yes |
| cfg            | None | 配置檔案 | 配置文件相關     | Yes |
| tracker        | botsort.yaml | 追蹤器配置 | 配置文件相關     | Yes |


## 2.4 訓練過程(3) — 模型架構初始化

```
Overriding model.yaml nc=80 with nc=26
...
Model summary: 365 layers, 43,649,886 parameters, 43,649,870 gradients, 165.5 GFLOPs
```

這部分顯示模型的架構資訊：
- 原始模型支援 80 類別（`nc=80`），但根據資料集配置，將其重設為 26 類別（手語字母 A-Z）。
- 模型總共有 365 層、43,649,886 個參數，且模型計算量為 165.5 GFLOPs。
    - GFLOPs 代表 Giga Floating Point Operations per Second，即每秒執行的十億次浮點運算次數。
- 中間省略的部分是模型結構的一覽表，請參考以下解釋。

### 2.4.1 YOLO 模型結構詳解

```txt
                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  8                  -1  3   4461568  ultralytics.nn.modules.block.C2f             [512, 512, 3, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  3   4723712  ultralytics.nn.modules.block.C2f             [1024, 512, 3]                
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  3   1247744  ultralytics.nn.modules.block.C2f             [768, 256, 3]                 
 16                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  3   4592640  ultralytics.nn.modules.block.C2f             [768, 512, 3]                 
 19                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  3   4723712  ultralytics.nn.modules.block.C2f             [1024, 512, 3]                
 22        [15, 18, 21]  1   5602846  ultralytics.nn.modules.head.Detect           [26, [256, 512, 512]]      
 ```

這段輸出來自 YOLOv8 模型的架構初始化部分，展示了模型的各層結構及其參數配置。每一橫列代表模型中的一層。以下是各欄位的解釋：

##### 欄位解釋
1. **from**：
    - 這欄顯示了當前層的輸入來自哪一層。例如，`-1` 表示來自上一層，`[-1, 6]` 表示來自上一層和第 6 層的輸出。
   
2. **n**：
   - 表示這一層中有多少次重複。例如，`1` 表示這一層只執行一次操作。

3. **params**：
   - 這一欄顯示了該層中可訓練的參數數量。例如，`1856` 表示這一層有 1,856 個參數。
   
4. **module**：
   - 這欄顯示了該層所使用的模組的類型或名稱。例如，`ultralytics.nn.modules.conv.Conv` 表示這是一個卷積層，`C2f` 是一種特定的模組，是 YOLOv8 中常見的模組之一。

5. **arguments**：
   - 這一欄列出了構建該層所需的參數或配置。例如，`[3, 64, 3, 2]` 表示這個卷積層的輸入通道數為 3，輸出通道數為 64，卷積核（kernel）大小為 3，步長（stride）為 2。

##### 舉例說明
讓我們來看幾個具體的例子來理解這些欄位：

- **第 0 行**：
  ```
  0  -1  1  1856  ultralytics.nn.modules.conv.Conv  [3, 64, 3, 2]  
  ```
  這表示第一層是一個**卷積層**（`Conv`），它的輸入來自上一層（這裡是模型的輸入），輸入通道數為 3（通常對應於 RGB 影像的 3 個通道），輸出通道數為 64，卷積核大小為 3，步長為 2。這層有 1856 個可訓練參數。

- **第 2 行**：
  ```
  2  -1  3  279808  ultralytics.nn.modules.block.C2f  [128, 128, 3, True]
  ```
  這層是一個重複 3 次的 **`C2f` 模組** ，輸入通道數和輸出通道數都是 128，卷積核大小為 3，並且啟用了某個布林參數（可能與是否啟用 shortcut 有關）。這層有 279,808 個可訓練參數。

- **第 22 行**：
  ```
  22  [15, 18, 21]  1  5602846  ultralytics.nn.modules.head.Detect  [26, [256, 512, 512]]
  ```
  這層是 YOLOv8 中的物件偵測頭（detection head），它接受第 15 層、第 18 層和第 21 層的輸出作為輸入，並且執行一次操作。這層有 5,602,846 個可訓練參數，並且配置了 26 個類別（例如對應 26 個手語字母），以及三個不同的特徵層（每個特徵層的通道數分別為 256、512 和 512）。

##### 總結
這段輸出詳細展示了 YOLOv8 模型的層級結構，每一層的輸入、類型、參數數量等資訊。這些資料對於了解模型的計算圖、調整模型架構、分析模型的複雜度，以及進一步優化模型都有很大的幫助。


## 2.5 訓練過程(4) — 載入預訓練權重

```txt
Transferred 589/595 items from pretrained weights
```

這代表成功載入了**預訓練權重**（pretrained weights），大部分權重都轉移到了新的模型中。

## 2.6 訓練過程(5) — 分散式訓練初始化

```txt
DDP: debug command ...
```

這部分表明使用的是**分散式資料平行化**（Distributed Data Parallel, DDP）來加速訓練，在多個 GPU 上運行。

## 2.7 訓練過程(6) — 啟動 TensorBoard 

```txt
TensorBoard: Start with 'tensorboard --logdir runs/detect/train2', view at http://localhost:6006/
```

這段提示你可以啟動 **TensorBoard** 來可視化訓練過程，通過指定的 URL 查看訓練進度。透過 Ngrok 的反向代理，指定的 URL 為 https://ca45-34-46-212-221.ngrok-free.app 。

## 2.8 訓練過程(7) — 自動混合精度（AMP）檢查

```txt
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
```

這部分顯示 YOLOv8 正在執行**自動混合精度**（Automatic Mixed Precision, AMP）檢查，以提高訓練效能並降低記憶體消耗，檢查順利通過。

AMP 是一種深度學習訓練技術，它通過在訓練過程中動態選擇不同精度的浮點數（如 FP16 和 FP32），來加速模型訓練，同時節省記憶體使用量。

## 2.9 訓練過程(8) — 資料載入和資料增強

```txt
train: Scanning /kaggle/working/datasets/American-Sign-Language-Letters-1/train/
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, ...
val: Scanning /kaggle/working/datasets/American-Sign-Language-Letters-1/valid/...
```

這部分的 `train` 和 `val` 顯示了訓練和驗證集的載入過程。

`albumentations` 那一行是應用於資料集的**圖像增強技術**（image augmentation），這是深度學習模型訓練中的一個常見策略，用來提高模型的泛化能力和防止過擬合。

YOLOv8 在訓練過程中應用了以下增強技術：
- Blur：對圖像應用**隨機模糊**。
- MedianBlur：使用**中值模糊**技術，這在降噪中非常有效。
- ToGray：將圖像轉換為**灰階**圖像。
- CLAHE：對比度受限的**自適應直方圖均衡化**（CLAHE），這是一種提高圖像對比度的技術。

## 2.10 訓練過程(9) — 優化器自動選擇

```txt
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', ...
```
這部分顯示 YOLOv8 自動選擇了最佳的**優化器**（optimizer）——[AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html)，忽略了手動設置的初始學習率和動量，根據模型和資料集特性來調整。

## 2.11 訓練過程(10) — 訓練啟動

```txt
Image sizes 800 train, 800 val
Using 4 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...
```

這部分顯示訓練正式開始：
- 使用的影像尺寸為 800x800 像素。
- 訓練過程中使用 4 個資料載入器（data loader）程序。
- 訓練結果會記錄到 `runs/detect/train2` 資料夾中。
- 訓練將持續 50 個 epoch。


### 11. 訓練報告
在 YOLOv8 的訓練過程中，每個 epoch 結束時都會引出一張表格來總結該次 epoch 的訓練結果和模型的性能評估。以下是這張表格中各個欄位的解釋：

#### 1. Epoch 行
```
      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      40/50      15.4G     0.6468     0.7795      1.187          5        800: 1
```

- **Epoch**：表示目前正在訓練的第幾個 epoch 以及總共的 epoch 數。例如，`40/50` 表示這是第 40 個 epoch，總共有 50 個 epoch。
  
- **GPU_mem**：當前使用的 GPU 記憶體量，這裡顯示為 `15.4G`，表示使用了 15.4 GB 的顯示卡記憶體（顯示記憶體）。

- **box_loss**：表示**邊界框回歸損失**（Bounding Box Regression Loss），這是模型在預測物件邊界框時的誤差。損失越小，表示模型預測的邊界框越接近真實值。

- **cls_loss**：表示**分類損失**（Classification Loss），這是模型在預測物件類別時的誤差。損失越小，表示模型在判斷物件類別時越準確。

- **dfl_loss**：表示**分佈焦點損失**（Distribution Focal Loss），這是 YOLOv8 使用的一種損失函數，用於改進邊界框的精度。

- **Instances**：每張影像中物件的平均數量，這裡顯示為 `5`，表示每張影像中平均有 5 個物件。

- **Size**：訓練影像的大小，這裡顯示為 `800: 1`，表示影像的尺寸為 800x800 像素，比例為 1:1。

#### 2. 評估行
```
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        144        144      0.941      0.874      0.952      0.761
```

- **Class**：這一行是指在所有類別上的平均結果。通常會列出各個類別的性能指標，但這裡匯總了所有類別的結果。

- **Images**：用於驗證的影像數量，這裡顯示為 `144`，表示使用了 144 張影像來評估模型的性能。

- **Instances**：驗證集中物件的總數，這裡顯示為 `144`，表示驗證集中有 144 個物件。

- **Box(P)**：精確率（Precision），表示模型預測的物件中有多少是正確的。這裡顯示為 `0.941`，即 94.1% 的預測是正確的。

- **R**：召回率（Recall），表示實際存在的物件中有多少被模型正確檢測出來。這裡顯示為 `0.874`，即 87.4% 的實際物件被檢測出來。

- **mAP50**：Mean Average Precision at IoU=0.50，這是物件偵測中的一個重要指標，表示在 IoU（Intersection over Union）= 0.50 門檻下的平均精度。這裡顯示為 `0.952`，即 95.2%。

- **mAP50-95**：Mean Average Precision at IoU=0.50:0.95，這是更嚴格的評估標準，表示在多個 IoU 門檻下的平均精度。這裡顯示為 `0.761`，即 76.1%。

#### 總結
這張表格總結了每個 epoch 訓練結束後的模型性能：

- **損失函數**（box_loss, cls_loss, dfl_loss）表明了模型的學習情況，損失越小越好。
- **精確率**（P）和**召回率**（R）反映了模型在預測中的精度和全面性。
- **mAP50** 和 **mAP50-95** 是物件偵測任務的核心指標，分別表示在不同 IoU 門檻下的平均精度，這些數值越高越好。

通過這些指標，你可以判斷模型在訓練過程中的表現如何，並且可以根據這些結果來調整模型和參數以獲得更好的性能。

In [ ]:
!ls {HOME}/runs/detect/train2/

#**Displaying the Confusion Matrix**

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/confusion_matrix.png', width=900)

#**Training and Validation Loss**

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/val_batch0_pred.jpg', width=600)

#**Validate Custom Model**

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train2/weights/best.pt data={dataset.location}/data.yaml

#**Inference with Custom Model**

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train2/weights/best.pt conf=0.25 source={dataset.location}/test/images

In [ ]:
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

#**Testing on a Demo Video**

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/best.pt conf=0.25 source='/content/demovideo/testvideo1.mp4'

#**Display the Demo Video**

In [ ]:
!rm "/content/result_compressed.mp4"

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = '/content/runs/detect/predict2/testvideo1.mp4'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)